# US Region vs Senate Party Timeline

legislators-historical.json obtained from https://github.com/unitedstates/congress-legislators

In [85]:
import json
from pprint import pprint

START_YEAR = 1914
END_YEAR = 2020
TERM_TYPE = 'sen'


def extract_info(members, output):
    for member in members:
        for term in member['terms']:
            if term['type'] == TERM_TYPE:
                start_year = int(term['start'][:4])
                end_year = int(term['end'][:4])
                
                for year in range(start_year, end_year):
                    if START_YEAR <= year <= END_YEAR:
                        state = term['state']
                        
                        if year not in output:
                            output[year] = {}
                        if state not in output[year]:
                            output[year][state] = []
                        output[year][state].append(term['party'])


def fill_data(filename, output):
    with open(filename) as data:
        extract_info(json.load(data), output)

In [86]:
senate_data = {}
fill_data('legislators-historical.json', senate_data)
fill_data('legislators-current.json', senate_data)

In [88]:
Region 1: Northeast
        Division 1: New England (Connecticut, Maine, Massachusetts, New Hampshire, Rhode Island, and Vermont)
        Division 2: Mid-Atlantic (New Jersey, New York, and Pennsylvania)
    Region 2: Midwest (Prior to June 1984, the Midwest Region was designated as the North Central Region.)[6]
        Division 3: East North Central (Illinois, Indiana, Michigan, Ohio, and Wisconsin)
        Division 4: West North Central (Iowa, Kansas, Minnesota, Missouri, Nebraska, North Dakota, and South Dakota)
    Region 3: South
        Division 5: South Atlantic (Delaware, Florida, Georgia, Maryland, North Carolina, South Carolina, Virginia, District of Columbia, and West Virginia)
        Division 6: East South Central (Alabama, Kentucky, Mississippi, and Tennessee)
        Division 7: West South Central (Arkansas, Louisiana, Oklahoma, and Texas)
    Region 4: West
        Division 8: Mountain (Arizona, Colorado, Idaho, Montana, Nevada, New Mexico, Utah, and Wyoming)
        Division 9: Pacific (Alaska, California, Hawaii, Oregon, and Washington)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 4)